In [76]:
import pandas as pd
import numpy as np

In [33]:
df = pd.read_csv('Realisierter_Stromverbrauch_201811010000_202411010000_Stunde.csv', 
                 sep= ';',
                 index_col= [0],
                 parse_dates= [0])

df = df.iloc[:,[1,2]]


# Replace '.' with '' (remove thousand separator) and ',' with '.' (convert to decimal point)
df.iloc[:, 0] = df.iloc[:, 0].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)

# Convert the column to float
df.iloc[:, 0] = df.iloc[:, 0].astype(float)

df_load = df['Gesamt (Netzlast) [MWh] Berechnete Auflösungen']

df_load


Datum von
01.11.2018 00:00     48879.5
01.11.2018 01:00     47312.5
01.11.2018 02:00    46093.75
01.11.2018 03:00     46710.5
01.11.2018 04:00    47624.25
                      ...   
31.10.2024 19:00     59605.0
31.10.2024 20:00    56859.25
31.10.2024 21:00     54040.5
31.10.2024 22:00     51405.0
31.10.2024 23:00    48110.75
Name: Gesamt (Netzlast) [MWh] Berechnete Auflösungen, Length: 52608, dtype: object

In [50]:
df = pd.read_csv('Merged_Data.csv', 
                 sep= ',',
                 index_col= [0],
                 parse_dates= [0])



df.columns

Index(['Datum bis_x', 'Gesamt (Netzlast) [MWh] Berechnete Auflösungen',
       'Residuallast [MWh] Berechnete Auflösungen',
       'Pumpspeicher [MWh] Berechnete Auflösungen_x', 'Datum bis_y',
       'Biomasse [MWh] Berechnete Auflösungen',
       'Wasserkraft [MWh] Berechnete Auflösungen',
       'Wind Offshore [MWh] Berechnete Auflösungen',
       'Wind Onshore [MWh] Berechnete Auflösungen',
       'Photovoltaik [MWh] Berechnete Auflösungen',
       'Sonstige Erneuerbare [MWh] Berechnete Auflösungen',
       'Kernenergie [MWh] Berechnete Auflösungen',
       'Braunkohle [MWh] Berechnete Auflösungen',
       'Steinkohle [MWh] Berechnete Auflösungen',
       'Erdgas [MWh] Berechnete Auflösungen',
       'Pumpspeicher [MWh] Berechnete Auflösungen_y',
       'Sonstige Konventionelle [MWh] Berechnete Auflösungen', 'Datum bis',
       'Deutschland/Luxemburg [€/MWh] Originalauflösungen',
       '∅ Anrainer DE/LU [€/MWh] Originalauflösungen',
       'Belgien [€/MWh] Originalauflösungen',
   

In [57]:
df_lo = df[['Gesamt (Netzlast) [MWh] Berechnete Auflösungen', 
         'Deutschland/Luxemburg [€/MWh] Originalauflösungen']]

df_po = df.iloc[:, 5:17]

df = pd.concat([df_lo, df_po], axis=1)

In [68]:
# Replace '-' with NaN
df = df.replace('-', float('nan'))

# Forward-fill to replace NaNs with the last valid value
df = df.ffill()

In [ ]:
# Replace '.' with '' (remove thousand separator) and ',' with '.' (convert to decimal point) for all columns
df = df.apply(lambda x: x.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False))

# Convert all columns back to float
df = df.astype(float)



In [78]:
def create_time_features(df, label=None):
    """
    Creates time series features from datetime index with sinusoidal encoding for cyclical features.
    """
    df = df.copy()
    
    # Ensure the index is a datetime object
    df['date'] = df.index
    
    # Create time-based features
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['day_name'] = df['date'].dt.day_name()
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['date_offset'] = (df['date'].dt.month * 100 + df['date'].dt.day - 320) % 1300
    
    # Encode cyclical features with sin and cos
    # Hour encoding
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    # Day of week encoding
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    # Month encoding
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # Day of year encoding
    df['dayofyear_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365)
    
    # Optionally add label column
    if label is not None:
        df['label'] = df[label]
    
    return df

In [79]:
df_features = create_time_features(df)

In [87]:
df_features.to_csv("merged_cleaned.csv")